# 랜덤화 블록, 라틴 정방 및 기타 설계

## 블록화 원리

실험에서 장애요인(Nuisance factor)을 제거하는 기법이다.  
장애요인은 실험자는 큰 관심이 없는 요인이지만 반응변수에 영향을 미칠 가능성이 다소 있으므로 장애요인이 반응변수에 전달하는 변동성은 최소화해야 한다.
- 예: 시간대, 날씨 등

다양한 산업계의 실험에서 블록화(Blocking)는 이미 반영되고 있고, 그렇지 않다고 해도 반영되어야 한다.  
블록화를 수행하지 않는 경우, 실험의 신뢰성을 저하시키고 불필요한 변동성을 반응변수에 남겨서 실험 결과를 왜곡할 수 있다.

장애요인이 알려져 있고 통제할 수 있다면 블록화를 사용한다.  
장애요인이 알려져 있으나 통제할 수 없는 경우 공분산 분석을 사용해 장애요인의 영향을 분석에서 제거할 수 있다.  
장애요인이 알려지지 않았고 통제할 수 없는 경우(잠재 변수), 랜덤화(Randomization)을 통해 실험 전반에 걸쳐 그 영향이 균형을 이루도록 한다.
- 영향을 없앨 수 없다면 장애요인이 모든 실험에서 골고루 펴지도록 설계하자는 것

__랜덤화 완전블록설계__
- 변동을 통제하기 위해 사용하는 설계 방법. 실험 단위는 블록이라는 그룹으로 나뉨.
- 각 블록이 모든 처리수준을 포함해야 하고 각 블록 안에 처리수준 하나당 한 개의 관측치가 존재한다.
- 각 블록 안의 각 처리수준에 해당하는 관측치는 랜덤하게 시행순서가 결정된다.
- 블록 내에서의 변동은 최소화하고 블록 간의 변동은 통제해 처리를 비교할 수 있다는 장점이 있다.

### ANOVA to RCBD

처리 평균이 같은지를 검정하는 것이 목적이지만, 그 전에 블록(장애요인)과 관련된 변동성을 제거하는 것이 선행되어야 한다.

RCBD의 통계적 모델

$$
y_{ij} = \mu + \tau_i + \beta_j + \epsilon_{ij}, \quad\begin{cases}
i=1,2,...,a \\
j=1,2,...,b
\end{cases}
$$
여기서:
- $\mu$: 전체 평균
- $\tau_i$: $i$번째 처리수준의 효과
- $\beta_j$: $j$번째 블록의 효과
- $\epsilon_{ij}$: $N(0,\sigma^2)$인 오차항

$$
y_{i.}=\sum^b_{j=1} y_{ij}, \quad i=1,2,...,a \\
y_{.j}=\sum^a_{i=1} y_{ij}, \quad j=1,2,...,b \\
y_{..} = \sum_{i=1}^a\sum_{j=1}^b y_{ij}=\sum_{i=1}^a y_{i.}=\sum_{j=1}^b y_{.j} \\
\bar{y}_{i.}=y_{i.}/b,\ \bar{y}_{.j}=y_{.j}/a,\ \bar{y}_{..}=y_{..}/N
$$

$$
\sum_{i=1}^a \tau_i=0,\ \sum_{j=1}^b \beta_j =0
$$

가설 설정

$$
\mu_i = \frac{1}{b} \sum_{j=1}^b (\mu+\tau_i + \beta_j)=\mu+\tau_i
$$

이므로:
- 모든 평균이 같다 $\to$ 모든 처리효과가 같다.

$$
H_0: \tau_1 = \tau_2 = \cdots = \tau_a \\
H_1: \text{At least one of }\mu_i \neq \mu_j \text{ is true.}
$$

제곱합 분해에서 항이 하나 더 추가:

$$
SS_T = SS_{Treatments} + SS_{Blocks} + SS_E
$$

| 변동원인  | 제곱합 $SS$       | 자유도 $df$         | 평균제곱 $MS$                            | $F_0$                                  |
|----------|---------------------|-----------------------|--------------------------------------------|------------------------------------------|
| 처리     | $SS_{\text{Treatments}}$ | $a - 1$              | $\frac{SS_{\text{Treatments}}}{a - 1}$  | $\frac{MS_{\text{Treatments}}}{MS_E}$  |
| 블록     | $SS_{\text{Blocks}}$    | $b - 1$              | $\frac{SS_{\text{Blocks}}}{b - 1}$      |                                          |
| 오차     | $SS_E$             | $(a - 1)(b - 1)$     | $\frac{SS_E}{(a - 1)(b - 1)}$           |                                          |
| 합계     | $SS_T$             | $N - 1$              |                                            |                                          |

$$
SS_T = \sum_{i=1}^{a} \sum_{j=1}^{b} y_{ij}^2 - \frac{y_{\cdot\cdot}^2}{N}
$$

$$
SS_{\text{Treatments}} = \frac{1}{b} \sum_{i=1}^{a} y_{i\cdot}^2 - \frac{y_{\cdot\cdot}^2}{N}
$$

$$
SS_{\text{Blocks}} = \frac{1}{a} \sum_{j=1}^{b} y_{\cdot j}^2 - \frac{y_{\cdot\cdot}^2}{N}
$$


#### 예시: 인공혈관

In [69]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols
import numpy as np

# 데이터 생성
data = {
    'Pressure': [8500, 8500, 8500, 8500, 8500, 8500,
             8700, 8700, 8700, 8700, 8700, 8700,
             8900, 8900, 8900, 8900, 8900, 8900,
             9100, 9100, 9100, 9100, 9100, 9100],
    'Batch': [1, 2, 3, 4, 5, 6,
             1, 2, 3, 4, 5, 6,
             1, 2, 3, 4, 5, 6,
             1, 2, 3, 4, 5, 6],
    'Y': [90.3, 89.2, 98.2, 93.9, 87.4, 97.9,
               92.5, 89.5, 90.6, 94.7, 87.0, 95.8,
               85.5, 90.8, 89.6, 86.2, 88.0, 93.4,
               82.5, 89.5, 85.6, 87.4, 78.9, 90.7]
}

# 데이터프레임 생성
df_long = pd.DataFrame(data)

# 이원배치 분산분석 모형 설정
model = ols('Y ~ C(Pressure) + C(Batch)', data=df_long).fit()

# ANOVA Table
anova_table = sm.stats.anova_lm(model, typ=1)

# 결과 출력
anova_table

,df,sum_sq,mean_sq,F,PR(>F)
C(Pressure),3.0,178.171250,59.390417,8.107077,0.001916
C(Batch),5.0,192.252083,38.450417,5.248666,0.005532
Residual,15.0,109.886250,7.325750,NaN,NaN


## 라틴정방설계(Latin Square Design)

랜덤화 완전블록설계는 알려져있고 통제 가능한 장애 변수로 인해 생기는 변동성을 제거해 실험에서 발생하는 잔차, 오차를 줄이도록 설계되었다.

라틴정방설계는 주로 두 가지의 장애 요인을 통제하면서 여러 가지 효과를 동시에 비교할 수 있도록 하는 설계이다.
- 라틴정방설계는 두 장애요인에 의한 변동성을 제거하기 위해 사용된다.

$$
y_{ijk}=\mu+\alpha + \tau_j + \beta_k + \epsilon_{ijk}\begin{cases}
i=1,2,...,p\\
j=1,2,...,p\\
k=1,2,...,p
\end{cases}
$$

$$
SS_T=SS_{Rows}+SS_{Columns}+SS_{Treatments}+SS_E
$$

| 변동원인   | 제곱합                                                                                   | 자유도            | 평균제곱                                   | F₀                                      |
|------------|------------------------------------------------------------------------------------------|-------------------|--------------------------------------------|-----------------------------------------|
| 처리       | $SS_{Treatments} = \frac{1}{p} \sum_{j=1}^{p} y_{\cdot j\cdot}^{2} - \frac{y_{\cdot \cdot}^{2}}{N}$ | $p-1$           | $\frac{SS_{Treatments}}{p-1}$           | $F_0 = \frac{MS_{Treatments}}{MS_{E}}$ |
| 행         | $SS_{Rows} = \frac{1}{p} \sum_{j=1}^{p} y_{i \cdot\cdot}^{2} - \frac{y_{\cdot \cdot}^{2}}{N}$ | $p-1$           | $\frac{SS_{Rows}}{p-1}$                 |                                         |
| 열         | $SS_{Columns} = \frac{1}{p} \sum_{j=1}^{p} y_{\cdot\cdot k}^{2} - \frac{y_{\cdot \cdot}^{2}}{N}$ | $p-1$           | $\frac{SS_{Columns}}{p-1}$              |                                         |
| 오류       | $SS_{E}\text{(차감에 의해)}$                                                                | $(p-2)(p-1)$    | $\frac{SS_{E}}{(p-2)(p-1)}$             |                                         |
| 합계       | $SS_T = \sum \sum \sum y_{ijk}^{2} - \frac{y_{\cdot \cdot \cdot}^{2}}{N}$                | $p^{2}-1$       |                                            |                                         |


In [5]:
# 데이터를 롱 포맷으로 변환하여 분산분석을 수행
import pandas as pd

# 각 처리 데이터를 열에 맞게 설정
long_format_data = {
    '원자재배치': [1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5],
    '조작자': [1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5],
    '처리': ['A', 'B', 'C', 'D', 'E', 'B', 'C', 'D', 'E', 'A', 'C', 'D', 'E', 'A', 'B', 'D', 'E', 'A', 'B', 'C', 'E', 'A', 'B', 'C', 'D'],
    '값': [24, 20, 19, 24, 24, 17, 24, 30, 27, 36, 18, 38, 26, 27, 21, 26, 31, 26, 23, 22, 22, 30, 20, 29, 31]
}

# 데이터프레임 생성
df_long = pd.DataFrame(long_format_data)

# 각 변동원에 대해 분산분석 수행 (처리, 원자재배치, 조작자)
import statsmodels.api as sm
from statsmodels.formula.api import ols

# ANOVA 모델 구성
model = ols('값 ~ C(처리) + C(원자재배치) + C(조작자)', data=df_long).fit()
anova_results = sm.stats.anova_lm(model, typ=2)

anova_results

,sum_sq,df,F,PR(>F)
C(처리),330.0,4.0,7.734375,0.002537
C(원자재배치),68.0,4.0,1.593750,0.239059
C(조작자),150.0,4.0,3.515625,0.040373
Residual,128.0,12.0,NaN,NaN
